# TCAD file exploration

We have received files from a client.  They are ....

# Shorten files for browsing

To shorten the files for browsing we can run a short shell script. This opens the zip that was received, and truncates each file at 100 lines long.

```{bash, eval=F}
# rm -rf shortened_appraisal_files
unzip original_data/Appraisal_Roll_History_1990.zip -d shortened_appraisal_files
find shortened_appraisal_files -name "*.TXT" -exec sed -i.full 100q {} \;
find shortened_appraisal_files -name "*.TXT.full" -exec rm {} \;
zip -r shortened_appraisal_files.zip shortened_appraisal_files
```

We can now attempt to load a shortened file using pandas

In [1]:
import pandas as pd

df = pd.read_csv("shortened_appraisal_files/Appraisal_Roll_History_1990_A/TCBC_SUM_1990_JURIS.TXT", sep = "|")

Challenge now is to use the *.TDF files to create tables.  I can think of two approaches.

1. The TDF files are SQL, so if those are fed to duckdb they should be able to create tables into which the TXT pipe-separated CSV files can be read.  There may be issues with the datatypes not matching (which would require mapping the current datatype definitions to duckdb datatypes by changing the words used to give the datatype to the columns).

2. Take the column names out of the TDF files and add them as the column names while reading the relevant CSV files into duckdb.  This would use duckdb's auto understanding of the column datatypes (so it would run, but it might guess wrongly and truncate or change data).

I think we should explore step 1 first.

## Creating tables using the TDF files

We have TDF files scattered through the _A and _B folders.  I think for now concentrate on the _A folder.

We can use python to read each TDF file separately, create the table and then try to load the matching TXT file.  A little guidance on how to process a directory structure of files using Path and glob here:
http://howisonlab.github.io/datawrangling/faq.html#get-data-from-filenames

In [1]:
import csv
from pathlib import Path
import duckdb

con = duckdb.connect('duckdb-file.db') #  string to persist to disk
cursor = con.cursor()

file_directory = 'shortened_appraisal_files/'
# limit_to_file = 'TCBC_SUM_1990_JURIS'
limit_to_file = '*' # all files

for filename in Path(file_directory).rglob(limit_to_file + '.TDF'):
    print(filename.parts)
    if "_A" in filename.parts[1]:
        table_name = "A_" + Path(filename).stem # e.g., A_TCBC_SUM_1990_JURIS
    else:
        table_name = "B_" + Path(filename).stem # e.g., B_TCBC_SUM_1990_JURIS
    
    # read .TDF file into string
    create_table_sql = Path(filename).read_text()
    # Need to alter table name to read in both _A and _B files
    create_table_sql = create_table_sql.replace(Path(filename).stem, table_name)
    
    # Here we have the table creation code in a string, so we can
    # swap datatypes out.
    # tried SMALLDATETIME --> DATETIME but was still giving errors
    # will need to fix this later.
    create_table_sql = create_table_sql.replace("SMALLDATETIME", "TEXT")

    # execute that SQL with duckdb, this should create the table
    cursor.execute(create_table_sql)

    # copy CSV into duckdb. CSV is the matching .TXT
    path_to_csvpipefile = Path(filename).with_suffix(".TXT")
    # duckdb copy documentation: https://duckdb.org/docs/sql/statements/copy.html
    query = f"COPY {table_name} FROM '{path_to_csvpipefile}' ( DELIMITER '|')"
    cursor.execute(query)

('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_CFOR.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_GRANT_EXMP.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_JURIS.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_JURIS_EXMP.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_LEGAL.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_SUSP.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_SUSP_INIT.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990_CFOR.TDF')
('shortened_appraisal_files', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990_GRANT_EXMP.TDF')
('shor

In [2]:
# setup from https://duckdb.org/docs/guides/python/jupyter.html
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
%sql duckdb:///duckdb-file.db

In [4]:
%%sql
SHOW TABLES

,name
0,A_TCBC_SUM_1990
1,A_TCBC_SUM_1990_CFOR
2,A_TCBC_SUM_1990_GRANT_EXMP
3,A_TCBC_SUM_1990_JURIS
4,A_TCBC_SUM_1990_JURIS_EXMP
5,A_TCBC_SUM_1990_LEGAL
6,A_TCBC_SUM_1990_SUSP
7,A_TCBC_SUM_1990_SUSP_INIT
8,A_TXBC_SUM_1990
9,A_TXBC_SUM_1990_CFOR


In [5]:
%%sql
SELECT * FROM A_TCBC_SUM_1990_JURIS 

,AcctNum,SufxId,TaxYear,Juris,Rate,JurisType,JurisCED,MdseVal,FrptVal,FFEVal,...,ExmpStatFlag,JurisPctFlag,FreeportFlag,FreeportStatus,AssessVal,TaxFrzVal,TaxBeforeFrz,GenFundTax,SinkFundTax,TotTax
0,0000000003,0000,1990,02,0.56950,CI,None,275,0,2923,...,None,None,None,None,4098.00,0.00,0.00,12.23,11.11,23.34
1,0000000003,0000,1990,03,0.40900,CO,None,275,0,2923,...,None,None,None,None,4098.00,0.00,0.00,16.76,0.00,16.76
2,0000000003,0000,1990,04,0.00010,CR,None,275,0,2923,...,None,None,None,None,4098.00,0.00,0.00,0.00,0.00,0.00
3,0000000003,0000,1990,08,1.64100,SD,None,275,0,2923,...,None,None,Y,None,4098.00,0.00,0.00,50.24,17.01,67.25
4,0000000007,0000,1990,01,1.26600,SD,None,25500,0,35000,...,None,None,Y,None,78000.00,0.00,0.00,836.55,150.93,987.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0000000056,0000,1990,04,0.00010,CR,None,5500,0,25000,...,None,None,None,None,138488.00,0.00,0.00,0.14,0.00,0.14
96,0000000056,0000,1990,68,0.05000,CC,None,5500,0,25000,...,None,None,Y,None,138488.00,0.00,0.00,69.24,0.00,69.24
97,0000000057,0000,1990,01,1.26600,SD,None,0,0,450,...,U,None,Y,None,450.00,0.00,0.00,4.83,0.87,5.70
98,0000000057,0000,1990,02,0.56950,CI,None,0,0,450,...,U,None,None,None,450.00,0.00,0.00,1.34,1.22,2.56
